In [1]:
import {InferenceSession, Tensor} from 'onnxruntime-node';
const ndarray = require('ndarray')
const ops = require('ndarray-ops')
const fs = require('fs')
const bisweb = require('biswebnode')



In [19]:
import {InferenceSession, Tensor, env} from 'onnxruntime-web';
const fs = require('fs')


// use an async context to call onnxruntime functions.
async function main() {
    try {

        // const float32Data = new Float32Array(160*256*256*1).fill(1);
        // const inputTensor = new Tensor("float32", float32Data, [160,256,256,1]);
        env.debug = true;
        env.logLevel = 'verbose';
        const float32Data = new Float32Array(211*224*224).fill(1);
        const inputTensor = new Tensor("float32", float32Data, [211,224,224,1]);
        // inputTensor.reshape([211,224,224,1])
        console.log(inputTensor.data)
        // data = imageDataToTensor(nvimage.img, [211,224,224,1])
        

        var session = await InferenceSession.create('../public/model/model.onnx');
        console.log(session.inputNames[0])

        // prepare feeds. use model input names as keys.
        var feeds = { input_1:inputTensor };


        // feed inputs and run
        var results = await session.run(feeds, ['conv2d_transpose_4']);

        // read from results
        const dataC = results;
        console.log(`data of result tensor 'c': ${dataC}`);

        return dataC

    } catch (e) {
        console.error(`failed to inference ONNX model: ${e}.`);
    }
}

var res = await main();
console.log(res)

Float32Array(10587136) [
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1,
  ... 10587036 more items
]
input_1
data of result tensor 'c': [object Object]
{
  conv2d_transpose_4: l {
    dims: [ 211, 224, 224, 1 ],
    type: 'float32',
    data: Float32Array(10587136) [
      0.002875823061913252,
      -0.0035482447128742933,
      0.0015134376008063555,
      -0.00024906962062232196,
      -0.0005567597690969706,
      -0.0010869965190067887,
      0.0001240931887878105,
      0.00011437403009040281,
      -0.00022227723093237728,
      0.0006404236191883683,
      0.00023755755682941526,
      0.00027328336727805436,
      0.0003408203483559191,
      0.0003292314358986914,
      -0.000867693976033479,
      -0.000025216

In [15]:
import {InferenceSession, Tensor} from 'onnxruntime-node';
const fs = require('fs')


// use an async context to call onnxruntime functions.
async function main() {
    try {

        // const float32Data = new Float32Array(160*256*256*1).fill(1);
        // const inputTensor = new Tensor("float32", float32Data, [160,256,256,1]);
        const float32Data = new Float32Array(211*224*224).fill(1);
        const inputTensor = new Tensor("float32", float32Data, [211,224,224,1]);
        // inputTensor.reshape([211,224,224,1])
        console.log(inputTensor.data)
        // data = imageDataToTensor(nvimage.img, [211,224,224,1])
        

        var session = await InferenceSession.create('../public/model/model.onnx');
        console.log(session.inputNames[0])

        // prepare feeds. use model input names as keys.
        var feeds = { input_1:inputTensor };


        // feed inputs and run
        var results = await session.run(feeds, ['conv2d_transpose_4']);

        // read from results
        const dataC = results;
        console.log(`data of result tensor 'c': ${dataC}`);

        return dataC

    } catch (e) {
        console.error(`failed to inference ONNX model: ${e}.`);
    }
}

var res = await main();
console.log(res)

Float32Array(10587136) [
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
  1, 1, 1, 1,
  ... 10587036 more items
]
input_1
data of result tensor 'c': [object Object]
{
  conv2d_transpose_4: l {
    dims: [ 211, 224, 224, 1 ],
    type: 'float32',
    data: Float32Array(10587136) [
      0.002875823061913252,
      -0.0035482447128742933,
      0.0015134376008063555,
      -0.00024906962062232196,
      -0.0005567597690969706,
      -0.0010869965190067887,
      0.0001240931887878105,
      0.00011437403009040281,
      -0.00022227723093237728,
      0.0006404236191883683,
      0.00023755755682941526,
      0.00027328336727805436,
      0.0003408203483559191,
      0.0003292314358986914,
      -0.000867693976033479,
      -0.000025216

In [7]:
function imageDataToTensor(data, dims){
  // 1a. Extract the R, G, and B channels from the data to form a 3D int array
  const [R, G, B] = new Array([], [], []);
  for (let i = 0; i < data.length; i += 4) {
    R.push(data[i]);
    G.push(data[i + 1]);
    B.push(data[i + 2]);
    // 2. skip data[i + 3] thus filtering out the alpha channel
  }
  ///console.log(R);
  //console.log(G);
  //console.log(B);
  // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
  const transposedData = R.concat(G).concat(B);

  // 3. convert to float32
  let i, l = transposedData.length; // length, we need this for the loop
  const float32Data = new Float32Array(dims[0]*dims[1]*dims[2]); // create the Float32Array for output
  for (i = 0; i < l; i++) {
    float32Data[i] = transposedData[i] / 255.0; // convert to float
  }

  const inputTensor = new Tensor("float32", float32Data, dims);
  return inputTensor;
}

In [20]:
let float32Data = new Float32Array(160*256*256*1).fill(0);
console.log(float32Data.length)
const newArray = Array.from({...float32Data, length:10587136}).fill(2)
float32Data.splice(0, newArray.length, ...newArray);
// console.log(float32Data.length)

4:13 - Property 'splice' does not exist on type 'Float32Array'. Did you mean 'slice'?


In [19]:
let a = [1,  2,  3,  4,  5]
let b = [10, 20, 30]

a.splice(0, b.length, ...b)

console.log(a) 

[ 10, 20, 30, 4, 5 ]


In [74]:


async function runModel(model, preprocessedData) {
  const start = new Date();
  try {
    const input = preprocessedData;
    const feeds = {float_input: input};
    // feeds[model.inputNames[0]] = preprocessedData;
    const outputData = await model.run(feeds);
    const end = new Date();
    const inferenceTime = (end.getTime() - start.getTime());
    const output = outputData[model.outputNames[0]];
    return [output, inferenceTime];
  } catch (e) {
    console.error(e);
    throw new Error();
  }
}

//The softmax transforms values to be between 0 and 1
function softmax(resultArray) {
  // Get the largest value in the array.
  const largestNumber = Math.max(...resultArray);
  // Apply exponential function to each result item subtracted by the largest number, use reduce to get the previous result number and the current number to sum all the exponentials results.
  const sumOfExp = resultArray.map((resultItem) => Math.exp(resultItem - largestNumber)).reduce((prevNumber, currentNumber) => prevNumber + currentNumber);
  //Normalizes the resultArray by dividing by the sum of all exponentials; this normalization ensures that the sum of the components of the output vector is 1.
  return resultArray.map((resultValue, index) => {
    return Math.exp(resultValue - largestNumber) / sumOfExp;
  });
}

const [res, time] =  await runModel(session, data);
var output = res.data;
var inferenceTime = time;
var results = softmax(Array.prototype.slice.call(output))

// var topResults = [];
// for (let i = 0; i < results.length; i++) {
//   if (results[i] > 0.3) {
//     topResults.push([classes[i] + ": " + results[i]]);
//   }
// }

console.log(inferenceTime);

TypeError: Only absolute URLs are supported
    at getNodeRequestOptions (/home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1305:9)
    at /home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1410:19
    at new Promise (<anonymous>)
    at fetch (/home/thuy/repo/deepsyence/node_modules/node-fetch/lib/index.js:1407:9)
    at Object.createSessionHandler (/home/thuy/repo/deepsyence/node_modules/onnxruntime-web/dist/ort-web.node.js:6:159912)
    at Function.create (/home/thuy/repo/deepsyence/node_modules/onnxruntime-common/dist/ort-common.node.js:6:6607)
    at async evalmachine.<anonymous>:5:17
    at async Object.execute (/home/thuy/.nvm/versions/node/v14.4.0/lib/node_modules/tslab/dist/executor.js:175:17)
    at async JupyterHandlerImpl.handleExecuteImpl (/home/thuy/.nvm/versions/node/v14.4.0/lib/node_modules/tslab/dist/jupyter.js:219:18)


In [ ]:

async function runModel(model, preprocessedData) {
    const start = new Date();
    try {
      const input = new ort.Tensor(new Float32Array(preprocessedData), [1, 380]);
      const feeds = {float_input: input};
      // feeds[model.inputNames[0]] = preprocessedData;
      const outputData = await model.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[model.outputNames[0]];
      return [output, inferenceTime];
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

In [ ]:
//The softmax transforms values to be between 0 and 1
function softmax(resultArray) {
  // Get the largest value in the array.
  const largestNumber = Math.max(...resultArray);
  // Apply exponential function to each result item subtracted by the largest number, use reduce to get the previous result number and the current number to sum all the exponentials results.
  const sumOfExp = resultArray.map((resultItem) => Math.exp(resultItem - largestNumber)).reduce((prevNumber, currentNumber) => prevNumber + currentNumber);
  //Normalizes the resultArray by dividing by the sum of all exponentials; this normalization ensures that the sum of the components of the output vector is 1.
  return resultArray.map((resultValue, index) => {
    return Math.exp(resultValue - largestNumber) / sumOfExp;
  });
}

In [75]:
const [res, time] =  await runModel(session, data);
var output = res.data;
var inferenceTime = time;
var results = softmax(Array.prototype.slice.call(output))

1:28 - Cannot find name 'runModel'.
1:37 - Cannot find name 'session'.
4:15 - Cannot find name 'softmax'.


In [ ]:
var topResults = [];
for (let i = 0; i < results.length; i++) {
  if (results[i] > 0.3) {
    topResults.push([classes[i] + ": " + results[i]]);
  }
}

console.log(topResults);